In [ ]:
from aiida import load_profile,orm, engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params
import aiidalab_widgets_base as awb
from settings import *


out = widgets.Output()

In [ ]:
def make_date_range(start:datetime, end:datetime, months:int)->dict:
    dates = {}
    x = start - relativedelta(months=1)
    end += relativedelta(months=1)
    while x < end:
        dates[datetime.strftime(x,'%Y-%m-%d')] = datetime.strftime(x+relativedelta(months=months-1),'%Y-%m-%d')
        x += relativedelta(months=months)
    return dates

# Inversion

### Import data

In [ ]:
style = {"description_width": "initial"}
address = widgets.Text(
            placeholder = 'Address to files',
            description="Remote address",
            style=style,
        )
def prepare_inspect():
    computer = orm.load_computer(label='daint')
    remote_path = orm.RemoteData(remote_path = address.value,
                             computer = computer)
    builder = INSPECT.get_builder()
    builder.remotes = {'a':remote_path,}
                      #'b':remote_path_2}
    return builder

btn_submit_1 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_1.btn_submit.button_style="info"
btn_submit_1.btn_submit.description="Launch import"

line_h = widgets.HTML(
            value="""<hr>
            """
        )
info_text = widgets.HTML(
            value="""Find for nc lise in stash folder of previous 
            Collect sensitivities calclations.
            """
        )

def prepare_inspect_cs():
    qb = orm.QueryBuilder()
    qb.append(COLLECT_SENS, tag="cs", 
              filters={"attributes.exit_status": 0}
              )
    qb.append(orm.RemoteStashFolderData, with_incoming="cs", project="*")
    builder = INSPECT.get_builder()
    directories_ = {f'test_{j}':i[0] for j,i in enumerate(qb.all())}
    if directories_:
        builder.remotes_cs = directories_
        return builder
    else:
        print('No files found.')

btn_submit_2 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect_cs,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_2.btn_submit.button_style="success"
btn_submit_2.btn_submit.description="Launch CS import"

acc = widgets.Accordion(
            children=[
                 widgets.VBox(
                    children=[
                address,
                btn_submit_1,
                line_h,
                info_text,
                btn_submit_2
                    ])
            ],
        )
acc.set_title(0, "Import")
acc.selected_index = None
acc

### Step 1

In [ ]:
sq = sens_query.SearchSens()
sq

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


### Step 3

In [ ]:
prepend_text_ = f'#SBATCH --constrain=mc\n'+\
                f'#SBATCH --account=em05\n'
              

In [ ]:
def prepare_inversion(s_date,e_date):
    
    inverstion_parameters = inv.construct_dict()
    filter_dict = get_dict_from_filter()
    extras_dict = {k:{x.description:x.value for x in v} for k,v in sq.site_extras.items()}

    for i in sq.list_info_obs:
        i.update(extras_dict[i['name']])
        i.update(filter_dict[i['name']])

    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})

    #other params
    inverstion_parameters.update({'para': sq.species.value,
                                'domain':sq.domain.value,
                                'transport.model':sq.model.value,
                                'dtm.start':s_date,
                                'dtm.end':e_date,
                                'res.dir':'./'})
    
    #test conversion
    inverstion_parameters = {
    key.replace(".", "_"): value for key, value in inverstion_parameters.items()
}
    print(inverstion_parameters)
    remotes_dictionary = {k[:-3].replace('-','_'):v 
                         for k,v in sq.list_remotes.items()}
    
    #builder starts
    builder = INVERSION.get_builder()
    builder.code = orm.load_code('inversion@daint')
    builder.remotes = remotes_dictionary
    builder.observations = sq.selected_obs
    builder.metadata.options.custom_scheduler_commands = prepend_text_
    builder.inv_params = orm.Dict(inverstion_parameters)
    builder.start_date = orm.Str(s_date)
    builder.end_date = orm.Str(e_date)
    builder.chunk = orm.Str(inv.chunk.value)
    builder.chunk_w = orm.Str(inv.chunk_w.value)
    return builder

def get_dict_from_filter():
    filter_dict  ={}
    for k,v in sq.site_filter.items():
        filter_={}
        for i in v.children[0].children[0].children:
            f = {}
            for j in range(4):
                f[i.children[j].description] = i.children[j].value
            filter_[i.children[0].value]=f
        filter_dict[k] = filter_
    return filter_dict
        

def run_inversions():

    text = """Submitted inversion from {0} to {1}. Click
        <a href=/apps/apps/aiidalab-widgets-base/notebooks/process.ipynb?id={2}
        target="_blank">here</a> to follow.<br>"""
    submision_text = widgets.HTML('')
    start = datetime.strptime(sq.date_range.value[:10], '%Y-%m-%d')
    end = datetime.strptime(sq.date_range.value[12:], '%Y-%m-%d')
    dates = make_date_range(start, end, 3)
    for s,e in dates.items():
        pass
        #process = engine.submit(prepare_inversion(s,e))
        #submision_text.value += text.format(s,e,process.pk)
    display(submision_text)


In [ ]:
button_inversion = widgets.Button(
    description='Submit inversion',
)
def on_click(b):   
    start = datetime.strptime(sq.date_range.value[:10], '%Y-%m-%d')
    end = datetime.strptime(sq.date_range.value[12:], '%Y-%m-%d')
    dates = make_date_range(start, end, 3)
    print(dates)
    with out:
        run_inversions()
       
button_inversion.on_click(on_click)
button_inversion